Import Packages & Libraries

In [153]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from datetime import datetime
reviewDataFrame = pd.DataFrame()

Set URL of album to parse and User Agent info of browser to spoof

In [154]:
#url = 'http://pitchfork.com/reviews/albums/21864-strangers/'
#url = 'http://www.pitchfork.com/reviews/albums/8396-simpatico/'
#url = 'http://pitchfork.com/reviews/albums/'
#url = 'http://www.pitchfork.com/reviews/albums/6332-rest-proof-clockwork/'
#url = 'http://pitchfork.com/reviews/albums/21673-teens-of-denial/'
#url = 'http://pitchfork.com/reviews/albums/21654-it-came-from-nyc/'
url = 'http://www.pitchfork.com/reviews/albums/yves-tumor-safe-in-the-hands-of-love/'
#url = 'http://pitchfork.com/reviews/albums/21980-fantasma/'
#url = 'http://pitchfork.com/reviews/albums/15996-lou-reed-metallica/'
#url = 'http://pitchfork.com/reviews/albums/21997-blonde-on-blonde-revisited/'
#url = 'http://pitchfork.com/reviews/albums/20882-tha-truth/'
ua ={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}

Get web page & print reponse code - 200 means the page was retrieved successfully

In [155]:
r = requests.get(url, headers=ua)
retrievalTime = datetime.now()
print(r.status_code)

200


Create BeautifulSoup object for the particular review

In [156]:
reviewSoup = BeautifulSoup(r.text, "html.parser")
print(reviewSoup.prettify())
reviewTitle = reviewSoup.find(attrs={"name":"og:title"})

<!DOCTYPE html>
<html lang="en">
 <head>
  <title data-react-helmet="true">
   Yves Tumor: Safe in the Hands of Love Album Review | Pitchfork
  </title>
  <meta content="width=device-width, initial-scale=1.0, user-scalable=no" data-react-helmet="true" name="viewport"/>
  <meta content="Pitchfork" data-react-helmet="true" name="og:site_name"/>
  <meta content="Yves Tumor’s latest album is a benchmark in experimental music. It is searing and borderless, music that is aware of oppressive confinement, and music with an intoxicating urge to be free." data-react-helmet="true" name="description"/>
  <meta content="https://pitchfork.com/reviews/albums/yves-tumor-safe-in-the-hands-of-love/" data-react-helmet="true" name="og:url"/>
  <meta content="Yves Tumor: Safe in the Hands of Love" data-react-helmet="true" name="og:title"/>
  <meta content="Yves Tumor’s latest album is a benchmark in experimental music. It is searing and borderless, music that is aware of oppressive confinement, and music w

Get BeautifulSoup Elements of Review Components

In [157]:
reviewIDElement = reviewSoup.find(attrs={"rel": "canonical"})
artistElement = reviewSoup.find(class_='artist-links')
albumsElement = reviewSoup.find_all(class_=re.compile('review-title'))
albumArtElement = reviewSoup.find_all(class_=re.compile('tombstone__art$'))
labelsElement = reviewSoup.find_all("li", class_='labels-list__item')
yearElement = reviewSoup.find_all("span", class_=re.compile('meta-year$'))
bnmElement = reviewSoup.find(class_='bnm-txt')
scoreElement = reviewSoup.find_all(class_='score')
writerElement = reviewSoup.find(class_=re.compile('display-name$'))
genreElement = reviewSoup.find(class_=re.compile('genre-list'))
publishDateElement = reviewSoup.find(class_='pub-date')
reviewContent = reviewSoup.find(class_='contents dropcap')

Review ID & Link

In [160]:
reviewLink = reviewIDElement['href']
print(reviewLink)
try:
    reviewID = int(reviewLink[37:reviewLink.find('-')])
except:
    reviewID = None
print(reviewID)

https://pitchfork.com/reviews/albums/yves-tumor-safe-in-the-hands-of-love/
None


Artist Name & ID

In [161]:
try:
    artist = artistElement.find("li")
    #print(artist)
    artistHref = artist.find("a")['href']
    artistName = artist.text
    artistID = int(artistHref[9:artistHref.find('-')])
except:
    artistName = artistElement.text
    artistID = 0
    
print(artistName)
print(artistID)

Yves Tumor
34005


Album Names

In [162]:
albumNames = [str(a.contents[0]) for a in albumsElement]
print(albumsElement)
print(albumNames)

[<h1 class="single-album-tombstone__review-title">Safe in the Hands of Love</h1>]
['Safe in the Hands of Love']


Album Art Links

In [163]:
albumArtLinks = [a.find("img")['src'] for a in albumArtElement]
print(albumArtLinks)

['https://media.pitchfork.com/photos/5b919e87f5a0aa15a0e21ee0/1:1/w_320/safeinthehands.jpg']


Record Labels

In [164]:
labelList = [l.contents[0] for l in labelsElement]
print(labelList)

['Warp']


Publish Date

In [165]:
timeStr = publishDateElement.contents[0]
publishDate = datetime.strptime(timeStr, '%B %d %Y')
print(type(publishDate))
print(publishDate)

<class 'datetime.datetime'>
2018-09-07 00:00:00


Release Year & Reissue Flag

In [166]:
yearTexts = [i.text for i in yearElement]
print(yearTexts)
ir = 1 if any( '/' in year for year in yearTexts) else 0
years = []
for y in yearTexts:    
    if '/' in y:
        isReissue = 1
        releaseYear = y[y.index('•') + 2:y.index('/')]
    else:
        isReissue = 0
        releaseYear = y[y.index('•') + 2:]
    
    if len(releaseYear) is 0:
        releaseYear = publishDate.year
    
    years.append(releaseYear)
    
print(years)
print(ir)

[' • 2018']
['2018']
0


Score & BNM/BNR tag

In [167]:
#score = float(scoreElement.contents[0])
scores = [s.contents[0] for s in scoreElement]
print(scores)
try:
    isBNR = 1 if 'reissue' in bnmElement.text else 0
    isBNM = 1 if 'music' in bnmElement.text else 0
except:
    isBNM = 0
    isBNR = 0
    
print(isBNM)
print(isBNR)

['9.1']
1
0


Writer

In [168]:
writer = str(writerElement.contents[0])
writerLink = writerElement['href']
print(writer)
print(writerLink)

Jayson Greene
/staff/jayson-greene/


Genres

In [93]:
if genreElement is not None:
    genreLis = genreElement.find_all('li')
    genreList = [g.find('a').contents[0] for g in genreLis]
else:
    genreList = []
    
print(genreList)

['Metal']


Review

In [94]:
review = str(reviewContent)

In [95]:
print(type(artistID))
print(type(artistName))
print(type(reviewID))
print(type(albumNames))
print(type(reviewLink))
print(type(albumArtLinks))
print(type(genreList))
print(type(labelList))
print(type(writer))
print(type(scores))
print(type(isBNM))
print(type(isBNR))
print(type(publishDate))
print(type(years))
print(type(retrievalTime))
print(type(review))
print(len(years))
print(len(albumNames))
print(len(albumArtLinks))
print(len(scores))

<class 'int'>
<class 'str'>
<class 'int'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'str'>
<class 'list'>
<class 'int'>
<class 'int'>
<class 'datetime.datetime'>
<class 'list'>
<class 'datetime.datetime'>
<class 'str'>
7
7
7
7


In [101]:
reviewList = []

for x in range(0, len(scores)):
    print(artistID)
    reviewDict = {'artistID':artistID}
    print(artistName)
    reviewDict['artistName'] = artistName
    print(reviewID)
    reviewDict['reviewID'] = reviewID
    print(albumNames[x])
    reviewDict['albumName'] = albumNames[x]
    print(reviewLink)
    reviewDict['reviewLink'] = reviewLink
    print(albumArtLinks[x])
    reviewDict['albumArtLink'] = albumArtLinks[x]
    print(years[x])
    reviewDict['year'] = years[x]
    print(genreList)
    reviewDict['genreList'] = genreList
    print(labelList)
    reviewDict['labelList'] = labelList
    print(writer)
    reviewDict['writer'] = writer
    print(writerLink)
    reviewDict['writerLink'] = writerLink
    print(scores[x])
    reviewDict['score'] = scores[x]
    print(isBNM)
    reviewDict['isBNM'] = isBNM
    print(isBNR)
    reviewDict['isBNR'] = isBNR
    print(publishDate)
    reviewDict['publishDate'] = publishDate
    print(retrievalTime)
    reviewDict['retrievalTime'] = retrievalTime
    print(reviewContent)
    reviewDict['reviewContent'] = reviewContent
    reviewList.append(reviewDict)

33541
White Zombie
21654
It Came From N.Y.C.
https://pitchfork.com/reviews/albums/21654-it-came-from-nyc/
https://media.pitchfork.com/photos/5929b4365e6ef9596932259c/1:1/w_320/50fa1f1a.jpg
2016
['Metal']
['Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group', 'Numero Group']
Evan Minsker
/staff/evan-minsker/
8.1
0
0
2016-06-07 00:00:00
2018-09-08 14:36:57.632907
<div class="contents dropcap"><p>“Shut up, Beavis, you’re ruining it.” That’s <a href="https://www.youtube.com/watch?v=p-0kgGVbDQQ">Butt-Head telling Beavis</a> to stop interrupting White Zombie’s “Thunder Kiss ‘65” video. The cartoon duo treated Rob Zombie and Sean Yseult’s band with deep reverence, and ironically enough, the clip is widely credited as the reason for White Zombie’s initial leap to multi-platinum success. It’s easy to see why the band captured the imaginations of millions of B

In [124]:
reviewList[0]

{'albumArtLink': 'https://media.pitchfork.com/photos/5929b4365e6ef9596932259c/1:1/w_320/50fa1f1a.jpg',
 'albumName': 'It Came From N.Y.C.',
 'artistID': 33541,
 'artistName': 'White Zombie',
 'genreList': ['Metal'],
 'isBNM': 0,
 'isBNR': 0,
 'labelList': ['Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group',
  'Numero Group'],
 'publishDate': datetime.datetime(2016, 6, 7, 0, 0),
 'retrievalTime': datetime.datetime(2018, 9, 8, 14, 36, 57, 632907),
 'reviewContent': <div class="contents dropcap"><p>“Shut up, Beavis, you’re ruining it.” That’s <a href="https://www.youtube.com/watch?v=p-0kgGVbDQQ">Butt-Head telling Beavis</a> to stop interrupting White Zombie’s “Thunder Kiss ‘65” video. The cartoon duo treated Rob Zombie and Sean Yseult’s band with deep reverence, and ironically enough, the clip is widely credited as the reason for

In [145]:
#reviewDataFrame = reviewDataFrame.append([review])
#reviewDataFrame = reviewDataFrame.append(reviewList[1], ignore_index=True)
for i in reviewList:
    reviewDataFrame = reviewDataFrame.append([i])

In [147]:
reviewDataFrame

,albumArtLink,albumName,artistID,artistName,genreList,isBNM,isBNR,labelList,publishDate,retrievalTime,reviewContent,reviewID,reviewLink,score,writer,writerLink,year
0,https://media.pitchfork.com/photos/5929b4365e6...,It Came From N.Y.C.,33541,White Zombie,[Metal],0,0,"[Numero Group, Numero Group, Numero Group, Num...",2016-06-07,2018-09-08 14:36:57.632907,"<div class=""contents dropcap""><p>“Shut up, Bea...",21654,https://pitchfork.com/reviews/albums/21654-it-...,8.1,Evan Minsker,/staff/evan-minsker/,2016
0,https://media.pitchfork.com/photos/5929b4385e6...,Gods on Voodoo Moon,33541,White Zombie,[Metal],0,0,"[Numero Group, Numero Group, Numero Group, Num...",2016-06-07,2018-09-08 14:36:57.632907,"<div class=""contents dropcap""><p>“Shut up, Bea...",21654,https://pitchfork.com/reviews/albums/21654-it-...,6.5,Evan Minsker,/staff/evan-minsker/,1985
0,https://media.pitchfork.com/photos/5929b43a13d...,Pig Heaven,33541,White Zombie,[Metal],0,0,"[Numero Group, Numero Group, Numero Group, Num...",2016-06-07,2018-09-08 14:36:57.632907,"<div class=""contents dropcap""><p>“Shut up, Bea...",21654,https://pitchfork.com/reviews/albums/21654-it-...,6.8,Evan Minsker,/staff/evan-minsker/,1986
0,https://media.pitchfork.com/photos/5929b43bb13...,Psycho-Head Blowout EP,33541,White Zombie,[Metal],0,0,"[Numero Group, Numero Group, Numero Group, Num...",2016-06-07,2018-09-08 14:36:57.632907,"<div class=""contents dropcap""><p>“Shut up, Bea...",21654,https://pitchfork.com/reviews/albums/21654-it-...,7.5,Evan Minsker,/staff/evan-minsker/,1987
0,https://media.pitchfork.com/photos/5929b43dc00...,Soul-Crusher,33541,White Zombie,[Metal],0,0,"[Numero Group, Numero Group, Numero Group, Num...",2016-06-07,2018-09-08 14:36:57.632907,"<div class=""contents dropcap""><p>“Shut up, Bea...",21654,https://pitchfork.com/reviews/albums/21654-it-...,8.4,Evan Minsker,/staff/evan-minsker/,1987
0,https://media.pitchfork.com/photos/5929b43e5e6...,Make Them Die Slowly,33541,White Zombie,[Metal],0,0,"[Numero Group, Numero Group, Numero Group, Num...",2016-06-07,2018-09-08 14:36:57.632907,"<div class=""contents dropcap""><p>“Shut up, Bea...",21654,https://pitchfork.com/reviews/albums/21654-it-...,6.0,Evan Minsker,/staff/evan-minsker/,1989
0,https://media.pitchfork.com/photos/5929b4405e6...,God of Thunder EP,33541,White Zombie,[Metal],0,0,"[Numero Group, Numero Group, Numero Group, Num...",2016-06-07,2018-09-08 14:36:57.632907,"<div class=""contents dropcap""><p>“Shut up, Bea...",21654,https://pitchfork.com/reviews/albums/21654-it-...,7.6,Evan Minsker,/staff/evan-minsker/,1989
